# OpenBugger
This notebook is a self-contained demo of the OpenBugger package that automatically bugs python code using LibCST

This plan for the notebook is to develop all the components necessary to construct a pipeline that
starting from a python script, it extracts its Concrete Syntax Tree and use it to apply a sequence of revertable syntactical code-mutation to automatically generate training data for debugging language models.

1. Tools to ensure that the sequence of modification is consistent and does not overwrite previously introduced mutations.
2. Bugger class to store the local context of multiple bugs that can be applied in a chain.
3. InverseTransformer class that is able to reverse the transformation of any other transformer sharing the same bugger context.
4. 5/12 example LibCST transformers that can each implement a different logical bug.  
5. Use the InverseTransformer to generate accurate debugging instructions to be used as training data for Large Language Models.

## LibCST

In [1]:
from libcst.codemod import CodemodContext, Codemod
from libcst.metadata import MetadataWrapper
from libcst import Equal, GreaterThanEqual, GreaterThan, CSTNode, Module
from typing import List
from copy import deepcopy
import libcst as cst
from libcst.codemod import CodemodContext, ContextAwareTransformer, ContextAwareVisitor
from libcst.metadata import BatchableMetadataProvider, PositionProvider, CodePosition, CodeRange
import libcst.matchers as m
import uuid

Useful LibCstDocs: 

https://libcst.readthedocs.io/en/latest/metadata.html#position-metadata

https://libcst.readthedocs.io/en/latest/_modules/libcst/metadata/position_provider.html#PositionProvider

https://libcst.readthedocs.io/en/latest/_modules/libcst/metadata/position_provider.html#WhitespaceInclusivePositionProvidingCodegenState


https://libcst.readthedocs.io/en/latest/parser.html

## PositionContextUpdater and is_modified
Because we want to apply potentially random changes to the code we need to introduce some simple helper methods, contained in the is_modified function that use the meta-data saved from transformers after modifyng a node to prevent any other transformer from applying further modifications to the either Node, its parent or its childrens. 

Since each node modification might introduce or remove code we also use the PositionContextUpdater to maintain the context scratch consistent after each mutation.

In [2]:
def is_parent_CodeRange(range_1: CodeRange, range_2: CodeRange) -> bool:
    """Return True if range_1 is a parent of range_2"""
    #for range_1 to be a parent of range_2 it needs to start before the start of range_2 and end after the end of range_2
    # let's check separately the two conditions and then merge them together
    # first condition
    start_before_start = (range_1.start.line < range_2.start.line) or (range_1.start.line == range_2.start.line and range_1.start.column< range_2.start.column)
    end_after_end = (range_1.end.line > range_2.end.line) or (range_1.end.line == range_2.end.line and range_1.end.column >= range_2.end.column)
    parent = start_before_start and end_after_end
    # print("range_1 is a parent of range_2",parent)
    return parent
def is_child_CodeRange(range_1: CodeRange, range_2: CodeRange) -> bool:
    """Return True if range_1 is a child of range_2"""
    # to check if range_1 is a child of range_2 we need to check that range_1 starts after the start of range_2 and ends before the end of range_2
    start_after_start = (range_1.start.line > range_2.start.line) or (range_1.start.line == range_2.start.line and range_1.start.column> range_2.start.column)
    end_before_end = (range_1.end.line < range_2.end.line) or (range_1.end.line == range_2.end.line and range_1.end.column <= range_2.end.column)
    children = start_after_start and end_before_end 
    # print("range_1 is a child of range_2",children)
    return children
def is_equal_Coderange(range_1: CodeRange, range_2: CodeRange) -> bool:
    """Return True if range_1 is equal to range_2"""
    equal = (range_1.start.line == range_2.start.line and range_1.start.column == range_2.start.column) and (range_1.end.line == range_2.end.line and range_1.end.column == range_2.end.column)
    # print("range_1 is equal to range_2",equal)
    return equal
    
def is_modified(node: CSTNode, meta_pos: CodeRange  , context: CodemodContext) -> bool:
    """
    Return True if the node has been modified by a transformer
    
    """
    # we cane use the is_ functions to check if the node has been modified by a transformer by checking if the context contains nodes that are either equal, children or parents of the node
    # print(meta_pos,[x["original_position"] for x in context.scratch.values()])
    equal = any([is_equal_Coderange(meta_pos, x["original_position"]) for x in context.scratch.values()])
    children = any([is_child_CodeRange(meta_pos, x["original_position"]) for x in context.scratch.values()])
    parents = any([is_parent_CodeRange(meta_pos, x["original_position"]) for x in context.scratch.values()])
    return equal or children or parents 


def save_modified(context: CodemodContext, meta_pos: CodeRange, original_node: CSTNode, updated_node:  CSTNode, author: str) -> None:
    """
    Save the modified node in the context.scratch
    """
    context.scratch[meta_pos.start] = {
                "modified": True, 
                "original_position": meta_pos,
                "original_node":original_node ,
                "updated_node":updated_node,
                "author":author
                }
    

class PositionContextUpdater(ContextAwareTransformer):
    METADATA_DEPENDENCIES = (PositionProvider,)
    def __init__(self, context: CodemodContext) -> None:
        self.context = context
        #init parent
        super().__init__(self.context)
    def on_visit(self, node: "CSTNode") -> bool:
        return True 
    def update_positions(self, meta_pos: CodePosition) -> None:
            already_modified  = [x for x in self.context.scratch.values() if meta_pos.start== x["original_position"].start]
            # print("already in scratch",[(x["original_position"].start, x["original_position"].end) for x in already_modified]) 
            # print("modified by",[(x["author"]) for x in already_modified])
            modified_keys = [x for x in self.context.scratch.keys() if meta_pos.start== self.context.scratch.get(x)["original_position"].start]
            key = modified_keys[0]
            #compute the delta between the end of the original node and the new one
            delta = meta_pos.end.column - self.context.scratch[key]["original_position"].end.column
            #update the scratch with the new position of the node
            self.context.scratch[key]["original_position"] = meta_pos
            #update the column position of all the nodes in the scratch that are on the same line
            for (k,v) in zip(list(self.context.scratch.keys()),list(self.context.scratch.values())):
                if v["original_position"].start.line == meta_pos.start.line and k.column > key.column:
                    new_start = CodePosition(line=v["original_position"].start.line,column=v["original_position"].start.column+delta)
                    new_end = CodePosition(line=v["original_position"].end.line,column=v["original_position"].end.column+delta)
                    v["original_position"]=CodeRange(start= new_start,end = new_end)
                    self.context.scratch[new_start] = self.context.scratch.pop(k)    
            # print("the delta is",delta)   
    def on_leave(self, node, updated_node):
        meta_pos=self.get_metadata(PositionProvider, node)
        meta_scratch = self.context.scratch.get(meta_pos.start,None)
        if meta_scratch and node.deep_equals(meta_scratch["updated_node"]):
            self.update_positions(meta_pos)
        return updated_node    
    def get_positions(self):
        return self.positions    

## InverseTransformer

In [3]:
class InverseTransformer(ContextAwareTransformer):
        """ A transformer that inverts the changes made by other transformers that share the same context"""
        METADATA_DEPENDENCIES = (PositionProvider, )
        def __init__(
            self,
            context: CodemodContext):
            super().__init__(context)
            self.id = None
       
        def transform_module_impl(self, tree: cst.Module) -> cst.Module:
            return tree.visit(self)
        def debug(self, tree: cst.Module,id) -> cst.Module:
            self.id = id
            return self.transform_module(tree)
        def invert_node(self, original_node:cst.CSTNode, updated_node: cst.CSTNode) ->   cst.CSTNode:
            meta_pos = self.get_metadata(PositionProvider, original_node)
            #only updates nodes that are not already in the scratch
            already_modified  = [x for x in self.context.scratch.values() if meta_pos.start== x["original_position"].start]
            # if already_modified:
                # print("already found a node modified by",already_modified[0]["author"])
                # print("current author is",self.id)
            if already_modified and already_modified[0]["author"] == self.id:
                # print("reverting to old node",meta_pos.start, meta_pos.end)
                old_node= self.context.scratch[meta_pos.start]["original_node"]
                self.context.scratch[meta_pos.start]["debugged_node"]=old_node
                updated_node=old_node
            return updated_node   
        def leave_ComparisonTarget(self, original_node:cst.ComparisonTarget, updated_node: cst.ComparisonTarget) -> None:
            return self.invert_node(original_node,updated_node)
        def leave_Assign(self, original_node:cst.Assign, updated_node: cst.Assign):
            return self.invert_node(original_node,updated_node)
        def leave_While(self, original_node:cst.While, updated_node: cst.While):
            return self.invert_node(original_node,updated_node) 
        def leave_Comparison(self, original_node:cst.Comparison, updated_node: cst.Comparison):
            return self.invert_node(original_node,updated_node)        
        def leave_Index(self, original_node:cst.Index, updated_node: cst.Index):
            return self.invert_node(original_node,updated_node)
        def leave_Slice(self, original_node:cst.Slice, updated_node: cst.Slice):
            return self.invert_node(original_node,updated_node)

## Bugger

In [4]:
class Bugger(Codemod):
    """ A CodeMod that allows to chain multiple Transformers sharing the same context and reverse them"""  
    def __init__(self, transformers: List[ContextAwareTransformer]) -> None:
        
        self.context = CodemodContext()
        Codemod.__init__(self,self.context)
        self.transformers = [transformer(self.context) for transformer in transformers]
        self.inverse = InverseTransformer(self.context)
        self.position_updater = PositionContextUpdater(self.context)
        #the context scratchpad has an entry ["modfied_nodes"] indexed by the start position of the modified_nodes 
        self.debug = False
        self.debug_steps = []
    def print(self):
        print("original_code")
        if not self.original or not self.tainted:
            print("Run bugger.appply(module) before print")
        else:    
            print(self.original.code)
            print("tainted_code")
            print(self.tainted.code)
            bugged = self.original.deep_equals(self.tainted)
            print("The result of deep_equals between the concrete syntax tree of the original and the bugged code is {}".format(bugged))
            print("Checking for bugs...")
            self.print_bugs()
            if not self.clean:
                print("Run bugger.apply(bugger.tainted,debug=True) before print")
            else:    
                print("Debugging...")
                print("clean_code")
                print(self.clean.code)
                print("Checking if the debugged code is equal to the original code..")
                diff = self.original.deep_equals(self.clean)
                print("The result of deep_equals between the concrete syntax tree of the original and debugged code is {}".format(diff))
           
    def print_bugs(self):
        for modified in self.context.scratch.values():
            bug_type = modified["author"]
            pos = modified["original_position"]
            start_line, start_column = pos.start.line, pos.start.column
            end_line, end_column = pos.end.line, pos.end.column
            print("The following Node has a bug of type {} starting at line {}, column {} and ending at line {}, column {}.".format(bug_type,start_line,start_column,end_line,end_column))
            bugged_code = self.tainted.code_for_node(modified["updated_node"])
            debugged_code = self.original.code_for_node(modified["original_node"])
            print("The bug can be fixed by substituting the bugged code-string <{}> with the following code-string <{}>".format(bugged_code,debugged_code))

    def apply(self, tree:Module,debug:bool=False) ->Module:
        self.debug=debug
        return self.transform_module(tree)
    def transform_module_impl(self, tree: Module) -> Module:
        if not self.debug:
            self.original = tree
        tainted = tree
        for transformer in self.transformers:
            if self.debug:
                tainted=self.inverse.debug(tainted,transformer.id)
                self.debug_steps.append(tainted)
                
            else:
                tainted = transformer.mutate(tainted)
            tainted = self.position_updater.transform_module(tainted) 
        if self.debug: 
            self.clean=self.debug_steps[-1]
        else:       
            self.tainted = tainted   

        return tainted
    
class TestTransformer(ContextAwareTransformer):
        """ a test transformer that checks if a node has been modified by another transformer and if not pretends to modify it"""
        METADATA_DEPENDENCIES = (PositionProvider, )
        def __init__(
            self,
            context: CodemodContext):
            super().__init__(context)
            self.id = f"{self.__class__.__name__}-{uuid.uuid4().hex[:4]}"
        def transform_module_impl(self, tree: cst.Module) -> cst.Module:
            return tree.visit(self)
        def mutate(self, tree: cst.Module,reverse: bool = False) -> cst.Module:
            return self.transform_module(tree)
        def taint_node(self, original_node:cst.CSTNode, updated_node: cst.CSTNode) ->   cst.CSTNode:
            meta_pos = self.get_metadata(PositionProvider, original_node)
            already_modified = is_modified(original_node,meta_pos,self.context)
            if not already_modified:
                # print("adding to scratch",meta_pos.start, meta_pos.end)
                updated_node = original_node
                save_modified(self.context,meta_pos,original_node,updated_node,self.id)
            return updated_node      
        def leave_ComparisonTarget(self, original_node:cst.ComparisonTarget, updated_node: cst.ComparisonTarget) -> None:
            return self.taint_node(original_node,updated_node)
        def leave_Assign(self, original_node:cst.Assign, updated_node: cst.Assign):
            return self.taint_node(original_node,updated_node)
        def leave_While(self, original_node:cst.While, updated_node: cst.While):
            return self.taint_node(original_node,updated_node)
        def leave_Comparison(self, original_node:cst.Comparison, updated_node: cst.Comparison):
            return self.taint_node(original_node,updated_node)
        def leave_Index(self, original_node:cst.Index, updated_node: cst.Index):
            return self.taint_node(original_node,updated_node)
        def leave_Slice(self, original_node:cst.Slice, updated_node: cst.Slice):
            return self.taint_node(original_node,updated_node)
        
def bugger_example(transformers,script):
    """ a method to automatically create an example of the bugger class applying the transformers to the script and visualizing the debugging process"""
    bugger = Bugger(transformers)
    # Parse the script into a CST
    module = cst.parse_module(script)
    tainted = bugger.apply(module)
    clean = bugger.apply(tainted,debug=True)
    bugger.print()

### Example Debugging Output Using the Test Tranformer

In [5]:
transformers = [TestTransformer]
# Get the script as a string it should have  while loop and take multiple lines
script = "while x < y[10]:\n\tprint(x)\n\tx = x[2] + 1+y[2:10]"
bugger_example(transformers,script)


original_code
while x < y[10]:
	print(x)
	x = x[2] + 1+y[2:10]
tainted_code
while x < y[10]:
	print(x)
	x = x[2] + 1+y[2:10]
The result of deep_equals between the concrete syntax tree of the original and the bugged code is True
Checking for bugs...
The following Node has a bug of type TestTransformer-9ee2 starting at line 1, column 12 and ending at line 1, column 14.
The bug can be fixed by substituting the bugged code-string <10> with the following code-string <10>
The following Node has a bug of type TestTransformer-9ee2 starting at line 3, column 7 and ending at line 3, column 8.
The bug can be fixed by substituting the bugged code-string <2> with the following code-string <2>
The following Node has a bug of type TestTransformer-9ee2 starting at line 3, column 16 and ending at line 3, column 20.
The bug can be fixed by substituting the bugged code-string <2:10> with the following code-string <2:10>
Debugging...
clean_code
while x < y[10]:
	print(x)
	x = x[2] + 1+y[2:10]
Checking if 

# Example Bugs 

In this section of the notebook we develop the 5/12 example bugs using LibCST ContextAwareTransformer that use the is_modified method to check if the node was already targeted by a mutation and then apply the mutation and save the mutation to the scratchpad using save_modified method.

The bugs are:

1. incorrect_comparison_operator - Done
2. comparison_swap - Done
3. forgetting_to_update_variable - Done
4. infinite_loop - Done
5. off_by_k_index - Done
6. incorrect_return_value
7. incorrect_boolean_operator
8. using_wrong_type_of_loop
9. using_loop_variable_outside_loop
10. using_variable_before_assignment
11. using_wrong_variable_scope
12. incorrect_use_of_exception_handling
13. incorrect_function_call

The current approach only allows for the scratchpad context to be passed as initialization to the transformers, therefore we use generator functions for the bugs that could take multiple inputs like which operator to swap. These transformers apply the bug to ALL the target instances they find, we will derive some wrapper in the later part of the notebook to control the number of bugs or target a specific code-range.

### Incorrect Comparison Operator
This bug takes as input two libcst comparison operators and swaps every instance of the first for the second

In [6]:
from libcst import (Equal, GreaterThanEqual, LessThan, GreaterThan, 
                    LessThanEqual, NotEqual, NotIn, In, Is, IsNot, Not, And, Or, Match)
import collections

def gen_ComparisonTargetTransfomer(op1, op2):
    """ A factory function that returns a ComparisonTargetTransformer that replaces all the occurences of op1 with op2"""

    str2op = dict([
    ('==', Equal),
    ('>=', GreaterThanEqual),
    ('>', GreaterThan),
    ('<', LessThan),
    ('=<', LessThanEqual),
    ('!=', NotEqual),
    ('not in', NotIn),
    ('in', In),
    ('is', Is),
    ('is not', IsNot),
    ('not', Not),
    ('and', And),
    ('or', Or),
    ('or', Or),])

    #check if the op1 and op2 are valid they can either be a string or a libcst comparisontarget operator class
    if op1 in str2op.keys():
        op1 = str2op[op1]
    elif op1 not in str2op.values():
        raise ValueError("op1 must be a string or a class") 
    if op2 in str2op.keys():
        op2 = str2op[op2]
    elif op2 not in str2op.values():
        raise ValueError("op2 must be a string or a class")
            
    class ComparisonTargetTransformer(ContextAwareTransformer):
        """ A transformer that replaces all the occurences of ComparisonTarget op1 with op2"""
        METADATA_DEPENDENCIES = (PositionProvider, )
        def __init__(
            self,
            context: CodemodContext):
            self.op1 = op1
            self.op2 = op2
            super().__init__(context)
            self.id = f"{self.__class__.__name__}-{uuid.uuid4().hex[:4]}"
        def transform_module_impl(self, tree: cst.Module) -> cst.Module:
            return tree.visit(self)
        def mutate(self, tree: cst.Module,reverse: bool = False) -> cst.Module:
            return self.transform_module(tree)
              
        def leave_ComparisonTarget(self, original_node:cst.ComparisonTarget, updated_node: cst.ComparisonTarget) -> None:
            meta_pos = self.get_metadata(PositionProvider, original_node)
            already_modified = is_modified(original_node,meta_pos,self.context)
            if not already_modified and original_node.operator.__class__ == self.op1: 
                updated_node = original_node.with_changes(operator=self.op2()) # OP2
                save_modified(self.context,meta_pos,original_node,updated_node,self.id)
            return updated_node
            
        def __repr__(self):
            return super().__repr__(self) + ':' + op1.__name__ +':' + op2.__name__
    
    return ComparisonTargetTransformer     

In [7]:
transformers = [gen_ComparisonTargetTransfomer('==','!=')]
# Get the script as a string
script = "x == 1 + 2 == 3 + 2 != 3 + 4 > 3"

bugger_example(transformers,script)

original_code
x == 1 + 2 == 3 + 2 != 3 + 4 > 3
tainted_code
x != 1 + 2 != 3 + 2 != 3 + 4 > 3
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type ComparisonTargetTransformer-6fbb starting at line 1, column 1 and ending at line 1, column 10.
The bug can be fixed by substituting the bugged code-string < != 1 + 2> with the following code-string < == 1 + 2>
The following Node has a bug of type ComparisonTargetTransformer-6fbb starting at line 1, column 10 and ending at line 1, column 19.
The bug can be fixed by substituting the bugged code-string < != 3 + 2> with the following code-string < == 3 + 2>
Debugging...
clean_code
x == 1 + 2 == 3 + 2 != 3 + 4 > 3
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


## Comparison Swap

In [8]:
class ComparisonSwapTransformer(ContextAwareTransformer):
    """ Swap the left and right side of a comparison, if there are moe than one comparison it swaps the first one"""    
    METADATA_DEPENDENCIES = (PositionProvider,)
    def __init__(self, context: CodemodContext):
        super().__init__(context)
        self.id = f"{self.__class__.__name__}-{uuid.uuid4().hex[:4]}"
    def mutate(self, tree: cst.Module,) -> cst.Module:
            return self.transform_module(tree)
    

    def leave_Comparison(self, original_node: cst.Comparison, updated_node: cst.Comparison) -> None:
        meta_pos = self.get_metadata(PositionProvider, original_node)
        #only updates nodes that are not already in the scratch
        already_modified = is_modified(original_node,meta_pos,self.context)
        
        comparisons = original_node.comparisons
        
        # only update the first comparison
        if not already_modified and len(comparisons) > 0:
            left = original_node.left
            lpar = original_node.lpar
            rpar = original_node.rpar
            first_comparison = comparisons[0]
            new_left = first_comparison.comparator
            first_comparison = cst.ComparisonTarget(
                operator=first_comparison.operator, 
                comparator=left
            )
            
            comparisons = list(comparisons)
            comparisons[0] = first_comparison
            comparisons = tuple(comparisons)
            updated_node = cst.Comparison(
                left=new_left,
                comparisons=comparisons,
                lpar=lpar,
                rpar=rpar
            )
            save_modified(self.context,meta_pos,original_node,updated_node,self.id)
        return updated_node        

In [9]:
transformers = [ComparisonSwapTransformer]
# Get the script as a string
script = "x == 1 + 2 == 3 + 2 != 3 + 4 > 3 "
bugger_example(transformers,script)

original_code
x == 1 + 2 == 3 + 2 != 3 + 4 > 3 
tainted_code
1 + 2 == x == 3 + 2 != 3 + 4 > 3 
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type ComparisonSwapTransformer-f11c starting at line 1, column 0 and ending at line 1, column 32.
The bug can be fixed by substituting the bugged code-string <1 + 2 == x == 3 + 2 != 3 + 4 > 3> with the following code-string <x == 1 + 2 == 3 + 2 != 3 + 4 > 3>
Debugging...
clean_code
x == 1 + 2 == 3 + 2 != 3 + 4 > 3 
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


## ForgettingToUpdateVariable

In [10]:
class ForgettingToUpdateVariableTransformer(ContextAwareTransformer):
        METADATA_DEPENDENCIES = (PositionProvider, )
        def __init__(
            self,
            context: CodemodContext):
            super().__init__(context)
            self.id = f"{self.__class__.__name__}-{uuid.uuid4().hex[:4]}"
        def transform_module_impl(self, tree: cst.Module) -> cst.Module:
            return tree.visit(self)
        def mutate(self, tree: cst.Module,reverse: bool = False) -> cst.Module:
            return self.transform_module(tree)
              
        def leave_Assign(self, original_node:cst.Assign, updated_node: cst.Assign) -> None:
            meta_pos = self.get_metadata(PositionProvider, original_node)
            #only updates nodes that are not already in the scratch
            already_modified = is_modified(original_node,meta_pos,self.context)
            if not already_modified:
                updated_node = original_node.with_changes(value=original_node.targets[0].target)
                save_modified(self.context,meta_pos,original_node,updated_node,self.id)
            return updated_node

In [11]:
transformers = [ForgettingToUpdateVariableTransformer]
# Get the script as a string
script = "while x == 1 + 2 == 3 + 2 != 3 + 4 > 3 : \n  y = 1 + 2"
bugger_example(transformers,script)

original_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3 : 
  y = 1 + 2
tainted_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3 : 
  y = y
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type ForgettingToUpdateVariableTransformer-7231 starting at line 2, column 2 and ending at line 2, column 7.
The bug can be fixed by substituting the bugged code-string <y = y> with the following code-string <y = 1 + 2>
Debugging...
clean_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3 : 
  y = 1 + 2
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


## Infinite While Loop

In [12]:
class InfiniteWhileTransformer(ContextAwareTransformer):
    METADATA_DEPENDENCIES = (PositionProvider,)
    def __init__(self, context: CodemodContext):
        super().__init__(context)
        self.id = f"{self.__class__.__name__}-{uuid.uuid4().hex[:4]}"
    def mutate(self, tree: cst.Module,reverse: bool = False) -> cst.Module:
            self.reverse=reverse
            return self.transform_module(tree)
    def leave_While(self, original_node: cst.While, updated_node: cst.While) -> None:
        meta_pos = self.get_metadata(PositionProvider, original_node)
        #only updates nodes that are not already in the scratch
        already_modified = is_modified(original_node,meta_pos,self.context)
        if not already_modified: 
            updated_node = cst.While(
                test=cst.Name("True"),
                body=original_node.body
            )
            save_modified(self.context,meta_pos,original_node,updated_node,self.id)
        return updated_node

In [13]:
transformers = [InfiniteWhileTransformer]
# Get the script as a string
script = "while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = 1 + 2"
bugger_example(transformers,script)

original_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
tainted_code
while True: 
  y = 1 + 2
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type InfiniteWhileTransformer-1910 starting at line 1, column 0 and ending at line 2, column 11.
The bug can be fixed by substituting the bugged code-string <while True: 
  y = 1 + 2
> with the following code-string <while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
>
Debugging...
clean_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


## OffByKIndex 

In [14]:
def gen_OffByKIndexTransformer(k):
    k=int(k)
    class OffByKIndexTransformer(ContextAwareTransformer):
            """ Uses the leave_Index method to add k to the index value"""
            METADATA_DEPENDENCIES = (PositionProvider,)
            def __init__(self, context: CodemodContext):
                super().__init__(context)
                self.id = f"{self.__class__.__name__}-{uuid.uuid4().hex[:4]}"
            def mutate(self, tree: cst.Module,) -> cst.Module:
                    return self.transform_module(tree)
            def leave_Index(self, original_node: cst.Index, updated_node: cst.Index) -> None:
                meta_pos = self.get_metadata(PositionProvider, original_node)
                #only updates nodes that are not already in the scratch
                already_modified = is_modified(original_node,meta_pos,self.context)
                if not already_modified:
                    # print("adding to scratch",meta_pos.start, meta_pos.end)
                    updated_node = original_node.with_changes(value=original_node.value.with_changes(value=str(int(original_node.value.value)+k)))
                    save_modified(self.context,meta_pos,original_node,updated_node,self.id)
                
                return updated_node    
            def leave_Slice(self, original_node: cst.Slice, updated_node: cst.Slice) -> None:
                meta_pos = self.get_metadata(PositionProvider, original_node)
                #only updates nodes that are not already in the scratch
                already_modified = is_modified(original_node,meta_pos,self.context)
                if not already_modified:
                    # print("adding to scratch",meta_pos.start, meta_pos.end)
                    lower = original_node.lower.value if original_node.lower is not None else None
                    upper = original_node.upper.value if original_node.upper is not None else None
                    step = original_node.step.value if original_node.step is not None else None
                    if lower is not None:
                        lower =cst.parse_expression(str(int(lower)+k))
                    if upper is not None:
                        upper = cst.parse_expression(str(int(upper)+k))
                    updated_node = original_node.with_changes(lower=lower, upper=upper, step=step)
                    save_modified(self.context,meta_pos,original_node,updated_node,self.id)
                return updated_node
    return OffByKIndexTransformer        

In [15]:
transformers = [gen_OffByKIndexTransformer(1)]
# Get the script as a string
script = "while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = 1 + 2\nx[1:2]\nx[1]"
bugger_example(transformers,script)

original_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
x[1:2]
x[1]
tainted_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
x[2:3]
x[2]
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type OffByKIndexTransformer-9ddf starting at line 3, column 2 and ending at line 3, column 5.
The bug can be fixed by substituting the bugged code-string <2:3> with the following code-string <1:2>
The following Node has a bug of type OffByKIndexTransformer-9ddf starting at line 4, column 2 and ending at line 4, column 3.
The bug can be fixed by substituting the bugged code-string <2> with the following code-string <1>
Debugging...
clean_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
x[1:2]
x[1]
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


## Chaining Multiple Bugs

Bugs can be chained in a sequence and they are executed in an order, bugs do not modify nodes that have already been modified or that either their children or parent has been modified. As an example we are going to all InfiniteWhiletransformer, OffByKIndexTransformer and ComparisonTargetTransfomer over the script:

while x == 1 + 2 == 3 + 2 != 3 + 4 > z[4]:  y = 1 + 2\nx[1:2]\nx[1]\nx[1]==3 

Comparison target and OffByKIndex only modify targets that are outside the while target statement.

In [16]:
transformers = [InfiniteWhileTransformer,gen_OffByKIndexTransformer(1),gen_ComparisonTargetTransfomer("==",">=")]
# Get the script as a string
script = "while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = 1 + 2\nx[1:2]\nx[1]\nx[1]==3"
bugger_example(transformers,script)

original_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
x[1:2]
x[1]
x[1]==3
tainted_code
while True: 
  y = 1 + 2
x[2:3]
x[2]
x[2] >= 3
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type InfiniteWhileTransformer-d444 starting at line 1, column 0 and ending at line 2, column 11.
The bug can be fixed by substituting the bugged code-string <while True: 
  y = 1 + 2
> with the following code-string <while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
>
The following Node has a bug of type OffByKIndexTransformer-bae8 starting at line 3, column 2 and ending at line 3, column 5.
The bug can be fixed by substituting the bugged code-string <2:3> with the following code-string <1:2>
The following Node has a bug of type OffByKIndexTransformer-bae8 starting at line 4, column 2 and ending at line 4, column 3.
The bug can be fixed by substituting the bugged code-string <2> with 